This notebook contains code for randomly selecting violations for evaluation by legal experts.

The second part of the work project involves using NLP and machine learning to offer suggestions of similar violations during a violation analysis. To evaluate the accuracy of the algorithms, a sample of infractions is selected at random and pairs are formed to assess experts for similarity on a scale of 1 (slightly similar infractions) to 10 (very similar infractions).

This assessment will be considered the standard against which the quality of the algorithms will be assessed.

In [4]:
# Load the necessary libraries

import nltk
import pandas as pd
import sqlalchemy as sal
from itertools import combinations

In [5]:
# Create the database connection

engine = sal.create_engine('mssql+pyodbc://LAPTOP-NNDGMEMB/beth?driver=ODBC+Driver+13+for+SQL+Server?Trusted_Connection=yes')
conn = engine.connect()
data = pd.read_sql_table('violations_content', conn)

In [6]:
# Remove the rows in which the document content is null.
# It happens when documents (documentoId) iniatilly loaded into 'violations_content'
# can't have their content retrieved (e.g. PDF that can't be parsed by PDFMiner)

data = data[~data['docContSplitted'].isna()]

In [7]:
# possible 'serieId' for documents that represent the initial document in a violation process

#serieId  serieNome                     
#344      REPRESENTAÇÃO - Eletrônica        887
#42       DEFESA                            405
#381      RECLAMAÇÃO - Atendimento SUSEP    151
#173      REPRESENTAÇÃO                      49
#341      AUTO - Infração Eletrônico         18
#61       RECLAMAÇÃO                         12
#45       DENÚNCIA                            1

initial_docs = [173,  344, 341, 381,  61,  45]

In [8]:
# filter the dataframe to keep only one document per violation and remove 'DEFESA'

data_orig = data.loc[data['serieId'].isin(initial_docs),:]
data_orig = data_orig.sort_values(by='documentoId', ascending=False)
data_orig = data_orig.drop_duplicates(subset='infracaoId', keep='first')

In [12]:
columnsToKeep = ['infracaoId', 'faltaDescr', 'dispositivoInfringido', 'docContSplitted']

In [13]:
dataToSelect = data_orig[columnsToKeep]

In [20]:
dataSelected = dataToSelect.sample(n=50, replace=False, random_state=123)

In [29]:
vioCombinations = pd.DataFrame(columns=['Inf A','Inf B'])

In [36]:
for i in combinations(dataSelected['infracaoId'],2):
    vioCombinations = vioCombinations.append(pd.Series(i, index=vioCombinations.columns), ignore_index=True)

In [38]:
vioCombinations.to_excel('vioCombinations.xlsx')

In [39]:
dataSelected.to_excel('vioWithData.xlsx')